In [ ]:
from src.stock_data import fetch_single_stock_data
from src.stock_features import prepare_data_for_ml, create_target_variable
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Define the parameters for your data pipeline
tickers_list = ['PG', 'KO', 'PEP', 'WMT', 'COST', '^GSPC']
start_date_str = '2010-01-01'
end_date_str = '2023-01-01'
output_filename = "consumer_stocks_final_engineered.csv"

# Make the single function call to run the entire pipeline
final_engineered_df = prepare_data_for_ml(
    tickers=tickers_list,
    start_date=start_date_str,
    end_date=end_date_str,
    output_engineered_csv=f'data/processed/{output_filename}'
)

final_engineered_df.head(3)



--- Starting Data Preparation Pipeline ---
Fetching data for 6 tickers from 2010-01-01 to 2023-01-01...

Discovered stock prefixes: ['COST', 'KO', 'PEP', 'PG', 'WMT', '^GSPC']

Processing features for stock prefix: COST
  - Calculating ATR for COST (window=14)...
  - Calculating RSI for COST (window=14)...
  - Calculating MACD for COST (fast=12, slow=26, signal=9)...

Processing features for stock prefix: KO
  - Calculating ATR for KO (window=14)...
  - Calculating RSI for KO (window=14)...
  - Calculating MACD for KO (fast=12, slow=26, signal=9)...

Processing features for stock prefix: PEP
  - Calculating ATR for PEP (window=14)...
  - Calculating RSI for PEP (window=14)...
  - Calculating MACD for PEP (fast=12, slow=26, signal=9)...

Processing features for stock prefix: PG
  - Calculating ATR for PG (window=14)...
  - Calculating RSI for PG (window=14)...
  - Calculating MACD for PG (fast=12, slow=26, signal=9)...

Processing features for stock prefix: WMT
  - Calculating ATR for 

,Close_COST,Close_KO,Close_PEP,Close_PG,Close_WMT,Close_^GSPC,High_COST,High_KO,High_PEP,High_PG,...,Close_^GSPC_lag5,Close_^GSPC_daily_return_lag1,Close_^GSPC_daily_return_lag3,Close_^GSPC_daily_return_lag5,^GSPC_RSI14_lag1,^GSPC_RSI14_lag3,^GSPC_RSI14_lag5,^GSPC_Volume_MA_Ratio_lag1,^GSPC_Volume_MA_Ratio_lag3,^GSPC_Volume_MA_Ratio_lag5
Date,,,,,,,,,,,,,,,,,,,,,
2010-03-16,43.420578,16.792852,42.058552,40.499805,13.515336,1159.459961,43.456221,16.846014,42.204966,40.735679,...,1140.449951,0.000452,0.004042,0.001713,84.225092,84.551764,80.135434,0.975939,1.101132,1.234186
2010-03-17,43.719986,16.836630,42.376842,40.805805,13.498439,1166.209961,43.812656,16.883538,42.383210,40.818553,...,1145.609985,0.007779,-0.000217,0.004525,88.120987,83.960195,82.581951,1.020683,1.155056,1.296198
2010-03-18,43.677216,16.871031,42.345013,40.627289,13.503260,1165.829956,43.791274,16.899175,42.478698,40.882284,...,1150.239990,0.005822,0.000452,0.004042,90.222369,84.225092,84.551764,1.149842,0.975939,1.101132


In [3]:
final_engineered_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3223 entries, 2010-03-16 to 2022-12-30
Columns: 290 entries, Close_COST to ^GSPC_Volume_MA_Ratio_lag5
dtypes: float64(284), int64(6)
memory usage: 7.2 MB
